In [ ]:
def train(X_train: pd.DataFrame, y_train: pd.DataFrame,
          model_directory_path: str = "resources") -> None:
    """
    Do your model training here.
    At each retrain this function will have to save an updated version of
    the model under the model_directiory_path, as in the example below.
    Note: You can use other serialization methods than joblib.dump(), as
    long as it matches what reads the model in infer().
    
    Args:
        X_train, y_train: the data to train the model.
        model_directory_path: the path to save your updated model
    
    Returns:
        None
    """
    #flush the memory
    ## For model construction
    #     alpha = 5
    # Num_hidden_limit = X_train.shape[0] / ( alpha * (X_train.shape[1] + 1) )
    # #alpha = 2 --> 800
    # #alpha = 5 --> 320
    # #alpha = 10 --> 160

    train_samples = X_train.shape[0]
    n_features = X_train.shape[1] - 2
    #can this fit in memory?
    batch_size = 74267
    #needs to be small enough that it can fit in memory. 2500 x 461 should be good
    batch_size = 2500

    ## do not include batch size when constructing input layer.
    model = tf.keras.models.Sequential([
        #tf.keras.layers.Flatten(input_shape = X_train.shape),
        tf.keras.layers.Input(shape = (None, train_samples, n_features), batch_size = batch_size, name = "Input_Layer"),
        tf.keras.layers.Dense(400, activation='swish'),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(300, activation='swish'),
        tf.keras.layers.Dropout(0.02),
        tf.keras.layers.Dense(200, activation='swish', activity_regularizer = tf.keras.regularizers.L2(0.01)),
        tf.keras.layers.Dropout(0.02),
        tf.keras.layers.Dense(100, activation='swish', activity_regularizer = tf.keras.regularizers.L2(0.01)),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(10, activation ='relu'),
        tf.keras.layers.Dense(1, activation = None)
    ])

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    my_callbacks = [ callback ]
    #if using KL Divergence as loss, don't add it as a metric
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3) ,
        #optimizer = tf.keras.optimizers.experimental.SGD( learning_rate = 0.001),
        #loss = tf.keras.losses.BinaryCrossentropy(from_logits = True) ,
        loss = tf.keras.losses.KLDivergence(),
        )

    # #Can do this if wanted
    # model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3) ,
    #     #loss = tf.keras.losses.BinaryCrossentropy(from_logits = True) ,
    #     loss = 'mse',
    #     metrics = [tf.keras.metrics.KLDivergence()]
    #     )

    # training the model
    print("training...")
    history = model.fit(
        x = X_train.iloc[:,2:],
        y = y_train.iloc[:,2:],
        epochs = 15,
        steps_per_epoch = int( np.ceil(train_samples / batch_size)),
        validation_split = 0.1,
        shuffle = False,
        workers = 8,
        use_multiprocessing = True,
        verbose = 1
        callbacks = my_callbacks
     )
    print("Training complete.")


    ## SAVE THE MODEL
    # make sure that the train function correctly save the trained model
    # in the model_directory_path
    model_pathname = Path(model_directory_path) / "model.joblib"
    print(f"Saving model in {model_pathname}")
    joblib.dump(model, model_pathname)


def infer(X_test: pd.DataFrame,
          model_directory_path: str = "resources") -> pd.DataFrame:
    """
    Do your inference here.
    This function will load the model saved at the previous iteration and use
    it to produce your inference on the current date.
    It is mandatory to send your inferences with the ids so the system
    can match it correctly.
    
    Args:
        model_directory_path: the path to the directory to the directory in wich we will be saving your updated model.
        X_test: the independant  variables of the current date passed to your model.

    Returns:
        A dataframe (date, id, value) with the inferences of your model for the current date.
    """

    # loading the model saved by the train function at previous iteration
    model = joblib.load(Path(model_directory_path) / "model.joblib")
    
    # creating the predicted label dataframe with correct dates and ids
    y_test_predicted = X_test[["date", "id"]].copy()
    y_test_predicted["value"] = model.predict(X_test.iloc[:, 2:])

    return y_test_predicted